# Change Detection (Segmentation) using TAO Visual ChangeNet-Segmentation Network

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. 

Train Adapt Optimize (TAO) Toolkit  is a Python based AI toolkit for taking purpose-built AI models and customizing them with your own data.

<img align="center" src="https://d29g4g2dyqv443.cloudfront.net/sites/default/files/akamai/TAO/tlt-tao-toolkit-bring-your-own-model-diagram.png" width="1080">

## What is a Visual ChangeNet Network?

Visual ChangeNet is a state of the art transformer-based Change Detection model. Visual ChangeNet is based on the Siamese Network, which is a class of neural network architectures containing two or more identical subnetworks. The training algorithm works by updating the parameters across all the sub-networks in tandem. In TAO, Visual ChangeNet supports two images as input where the end goal is to classify or segment the change between the "golden or reference" image and the "test" image. TAO supports the [FAN](https://arxiv.org/abs/2204.12451) backbone network for both Visual ChangeNet architectures. For more details about training FAN backbones, see the Classification PyTorch notebook.

In TAO, two different types of Change Detection networks are supported to detect the difference between the two input images: 

* Visual ChangeNet-Segmentation for segmentation 
* Visual ChangeNet-Classification for classification  

In this notebook, the Visual ChangeNet-Segmentation model is leveraged to demonstrate change detection by segmenting the changed pixels between the two input images. 

### Sample Inference from the Visual ChangeNet-Segmentation Model

The following is a sample inference output for the Visual ChangeNet-Segmentation model on the MVTec Bottle class. The model takes in two input images (test and golden reference) and outputs a segmentation mask showing the defective region on an object in the MVTec Anomaly Dataset.


<img align="center" src="https://github.com/vpraveen-nv/model_card_images/blob/main/cv/purpose_built_models/visual_changenet/mvtech_example.png?raw=true" width="1080">



## Learning Objectives

In this notebook, you learn how to leverage TAO to:

* Train a Visual ChangeNet-Segmentation Model for defect detection on objects from the MVTec Anomaly Dataset.
* Evaluate the trained model.
* Run Inference on the trained model.
* Export the trained model to a .onnx file (encrypted ONNX model) for deployment to DeepStream, TensorRT, or Triton.

At the end of this notebook, you generate a trained and optimized `visual_changenet` model, 
which you can deploy with this [end-to-end sample](https://github.com/NVIDIA-AI-IOT/tao-toolkit-triton-apps) with Triton.


## Table of Contents

This notebook shows an example usecase of the Siamese Network using the Train Adapt Optimize (TAO) Toolkit.

0. [Set up env variables and map drives](#head-0)
1. [Installing the TAO launcher](#head-1)
2. [Prepare dataset](#head-2)
3. [Download the pretrained model from NGC](#head-3)
4. [Provide training specification](#head-4)
5. [Run TAO training](#head-5)
6. [Evaluate trained models](#head-6)
7. [Inferences](#head-7)
8. [Deploy](#head-8)


## 0. Set up env variables and map drives <a class="anchor" id="head-0"></a>

The TAO launcher uses Docker containers and **for our data and results directory to be visible to Docker, they need to be mapped**. The launcher can be configured using the config file `~/.tao_mounts.json`. Apart from the mounts, you can also configure additional options such as, the Environment Variables and amount of Shared Memory available to the TAO launcher. <br>

`IMPORTANT NOTE:` The code below creates a sample `~/.tao_mounts.json`  file. It maps directories in which the data, specs, results, and cache information is saved. You must configure it for your specific case so that these directories are correctly visible to the Docker container.


In [ ]:
import os

# Please define this local project directory that needs to be mapped to the TAO docker session.
%env LOCAL_PROJECT_DIR=FIXME

os.environ["HOST_DATA_DIR"] = os.path.join(os.getenv("LOCAL_PROJECT_DIR", os.getcwd()), "data", "changenet")
os.environ["HOST_RESULTS_DIR"] = os.path.join(os.getenv("LOCAL_PROJECT_DIR", os.getcwd()), "changenet", "results")
os.environ["HOST_MODEL_DIR"] = os.path.join(os.getenv("LOCAL_PROJECT_DIR", os.getcwd()), "changenet", "model")

os.environ["HOST_SPECS_DIR"] = os.path.join(
    os.getenv("NOTEBOOK_ROOT", os.getcwd()),
    "specs"
)

In [ ]:
! mkdir -p $HOST_DATA_DIR
! mkdir -p $HOST_SPECS_DIR
! mkdir -p $HOST_RESULTS_DIR
! mkdir -p $HOST_MODEL_DIR

In [ ]:
# Mapping up the local directories to the TAO Docker.
import json
import os
mounts_file = os.path.expanduser("~/.tao_mounts.json")
tlt_configs = {
   "Mounts":[
       # Mapping the data directory
       {
           "source": os.environ["LOCAL_PROJECT_DIR"],
           "destination": "/workspace/tao-experiments"
       },
       {
           "source": os.environ["HOST_DATA_DIR"],
           "destination": "/data"
       },
       {
           "source": os.environ["HOST_MODEL_DIR"],
           "destination": "/model"
       },
       {
           "source": os.environ["HOST_SPECS_DIR"],
           "destination": "/specs"
       },
       {
           "source": os.environ["HOST_RESULTS_DIR"],
           "destination": "/results"
       }
   ],
   "DockerOptions": {
        "shm_size": "16G",
        "ulimits": {
            "memlock": -1,
            "stack": 67108864
         }
   }
}
# Writing the mounts file.
with open(mounts_file, "w") as mfile:
    json.dump(tlt_configs, mfile, indent=4)

In [ ]:
!cat ~/.tao_mounts.json

## 1. Installing the TAO launcher <a class="anchor" id="head-1"></a>
The TAO launcher is a Python package distributed as a Python Wheel listed in PyPI. Install the launcher by executing the following cell.

TAO Toolkit recommends that you run the TAO launcher in a virtual env with Python 3.7.0. Follow the instruction in this [page](https://virtualenvwrapper.readthedocs.io/en/latest/install.html) to set up a Python virtual env using the `virtualenv` and `virtualenvwrapper` packages. After you setup the virtualenvwrapper, set the version of Python to be used in the virtual env by using the `VIRTUALENVWRAPPER_PYTHON` variable. To run it:

```sh
export VIRTUALENVWRAPPER_PYTHON=/path/to/bin/python3.x
```
where x >= 7 and <= 8

Perform this step first and then launch the Notebook from the virtual environment. In addition to installing TAO Python package, you must meet the following software requirements:
* python >=3.7.0 < 3.8.x
* docker-ce > 19.03.5
* docker-API 1.40
* nvidia-container-toolkit > 1.3.0-1
* nvidia-container-runtime > 3.4.0-1
* nvidia-docker2 > 2.5.0-1
* nvidia-driver > 525+

After installing the pre-requisites, log in to the Docker registry nvcr.io using the following command:

```sh
docker login nvcr.io
```

Enter a username and password. The username is `$oauthtoken` and the password is the API key generated from `ngc.nvidia.com`. Follow the instructions in the [NGC setup guide](https://docs.nvidia.com/ngc/ngc-overview/index.html#generating-api-key) to generate your own API key.

The TAO Toolkit recommends that you run the TAO launcher in a virtual env with Python >=3.7.0. Follow these [instructions](https://virtualenvwrapper.readthedocs.io/en/latest/install.html) to set up a Python virtual env using the virtualenv and virtualenvwrapper packages.

In [ ]:
# SKIP this step IF you have already installed the TAO launcher.
!pip3 install nvidia-tao

In [ ]:
# View the versions of the TAO launcher
!tao info --verbose

## 2. Prepare a Class From The MVTech Anomaly dataset <a class="anchor" id="head-2"></a>

TAO Visual ChangeNet-Segmentation uses a custom dataset format. The sections below walk through this format, specifically the dataset structure and the files required.

### Visual ChangeNet-Segmentation data format

Visual ChangeNet-Segmentation expects directories of images and mask files in the dataset root directory. The image directories consist of
the golden image directory (pre-change images) and the test image directory (post-change image) to compare against the change mask images with pixel level change masks.  


     |--dataset_root:
          |--A
               |--image1.jpg
               |--image2.jpg
          |--B
               |--image1.jpg
               |--image2.jpg
          |--label
               |--image1.jpg
               |--image2.jpg      
          |--list
               |-- train.txt
               |-- val.txt
               |-- test.txt
               |-- predict.txt

Here's a description of the structure:

* The ``dataset_root`` directory contains the following:
    * ``A``: Contains post-change test images.
    * ``B``: Contains pre-change golden reference images.
    * ``label``: Contains ground truth segmentation change masks.
    * ``list``: Contains .txt files for each dataset split, as described in the  section below. 

#### List Files


Visual ChangeNet-Segmentation dataloader expects the ``label`` directory to contain ``.txt`` files for each of the dataset splits [train, validation, test, inference]. 
A Visual ChangeNet-Segmentation label file is a ``.txt`` file containing all file names for the particular split.


| ``image_names``   | 
| :---------------- |
| ``file_name.png`` |

* ``image_names``: The names of images. Image names must be the same for test images and their corresponding reference and mask images. 

The following sample label file corresponds to the sample directory structure as described.


| ``image_names``   | 
| :---------------- |
| ``image1.png``    |
| ``image3.png``    |
| ``image2.png``    |

Note: Each test image (inside directory ``A``) must have a reference image (inside directory ``B``)
and a segmentation change map (inside directory ``label``) with the same name for the dataloader to map them correctly.

# Getting The MVTech Dataset

The MVTech Anomaly dataset provides images of 15 different objects. For each object, the dataset contains non-defective images (golden images) and defective images with a pixel perfect segmentation mask of the defect region. The cells below convert the dataset from any of the 15 objects into the format expected by ChangeNet. 

* To run this example, go to https://www.mvtec.com/company/research/datasets/mvtec-ad.
* Follow the prompts to the download page, right click on any of the object datasets, and copy the link. 
* Do not copy the link for the whole dataset download because this Notebook only trains on a single object type. 
* Paste the link in the FIXME below.

### 2.1 Download the dataset <a class="anchor" id="head-2-1"></a>

In [ ]:
#Download the data
import os
MVTEC_AD_OBJECT_DOWNLOAD_URL = "FIXME"
mvtec_object = MVTEC_AD_OBJECT_DOWNLOAD_URL.split("/")[-1].split(".")[0]
os.environ["URL_DATASET"]=MVTEC_AD_OBJECT_DOWNLOAD_URL
os.environ["MVTEC_OBJECT"]=mvtec_object
!if [ ! -f $HOST_DATA_DIR/$MVTEC_OBJECT.tar.xz ]; then wget $URL_DATASET -O $HOST_DATA_DIR/$MVTEC_OBJECT.tar.xz; else echo "image archive already downloaded"; fi

### 2.2 Verify the downloaded dataset <a class="anchor" id="head-2-2"></a>

In [ ]:
# Check the dataset is present
!mkdir -p $HOST_DATA_DIR
!if [ ! -f $HOST_DATA_DIR/$MVTEC_OBJECT.tar.xz ]; then echo 'Dataset tar file not found, please download.'; else echo 'Found Dataset tar file.';fi

In [ ]:
# unpack 
!tar xf $HOST_DATA_DIR/$MVTEC_OBJECT.tar.xz -C $HOST_DATA_DIR
!chmod -R 755 $HOST_DATA_DIR/$MVTEC_OBJECT

### 2.3 Format the dataset <a class="anchor" id="head-2-3"></a>

In [ ]:
!pip install Pillow

In [ ]:
import random 
import shutil 
from PIL import Image
os.environ["HOST_DATA_DIR"] = os.path.join(os.environ["LOCAL_PROJECT_DIR"], "data", "changenet")
formatted_dir = f"formatted_{mvtec_object}_dataset"

DATA_DIR = os.environ["HOST_DATA_DIR"]
os.environ["FORMATTED_DATA_DIR"] = formatted_dir

#setup dataset folders in expected format 
formatted_path = os.path.join(DATA_DIR, formatted_dir)
a_dir = os.path.join(formatted_path, "A")
b_dir = os.path.join(formatted_path, "B")
label_dir = os.path.join(formatted_path, "label")
list_dir = os.path.join(formatted_path, "list")

#Create the expected folders
os.makedirs(formatted_path, exist_ok=True)
os.makedirs(a_dir, exist_ok=True)
os.makedirs(b_dir, exist_ok=True)
os.makedirs(label_dir, exist_ok=True)
os.makedirs(list_dir, exist_ok=True)

In [ ]:
#Setup paths to image folders 
test_dir = os.path.join(DATA_DIR, mvtec_object, "test") #contains defect and good images 
ground_truth_dir = os.path.join(DATA_DIR, mvtec_object, "ground_truth") #contains masks for all defect images
no_defect_dir = os.path.join(DATA_DIR, mvtec_object, "train", "good") #contains only good images 

#Determine defect classes and image size
mvtec_object_classes = os.listdir(test_dir)
image_name = os.listdir(no_defect_dir)[0] #get an image to determine size
mvtec_img_size = Image.open(os.path.join(no_defect_dir,image_name)).size


#combine images of different defects into folder 'A'
for class_dir in os.listdir(test_dir):
    for image in os.listdir(os.path.join(test_dir,class_dir)):
        shutil.copy(os.path.join(test_dir,class_dir,image), os.path.join(a_dir, f"{class_dir}_{image}"))

#combine ground truth images of different defects into folder 'label'
for class_dir in os.listdir(test_dir):
    for image in os.listdir(os.path.join(test_dir,class_dir)):
        if class_dir == "good": #make mask for good images
            good_mask = Image.new(mode="L", size=mvtec_img_size, color=(0))
            good_mask.save(os.path.join(label_dir, f"{class_dir}_{image}"))
        else: #dataset provides masks for defective images 
            image_name = os.path.splitext(image)[0]
            image_ext = os.path.splitext(image)[1]
            src_image = image_name + "_mask" + image_ext
            shutil.copy(os.path.join(ground_truth_dir,class_dir,src_image), os.path.join(label_dir, f"{class_dir}_{image}"))

            
#Split non defect images to use some for golden samples and some for training 
golden_split = 0.05 #percentage of good samples from the train set to use as golden images. 
no_defect_images = os.listdir(no_defect_dir)
no_defect_split = int(len(no_defect_images) * golden_split)
golden_ref_images = no_defect_images[0:no_defect_split] #taking a % of good images to serve as golden references
no_defect_train_images = no_defect_images[no_defect_split:] #adding the rest to the training set 

#Add good images not used for golden samples into the train set 
for i, image in enumerate(no_defect_train_images):
    dest_image = "good_2_" + image
    shutil.copy(os.path.join(no_defect_dir, image), os.path.join(a_dir, dest_image))
    #also add the mask
    good_mask = Image.new(mode="L", size=mvtec_img_size, color=(0))
    good_mask.save(os.path.join(label_dir, dest_image))
    
#Setup the golden reference images 
for i, image in enumerate(os.listdir(a_dir)):
    golden_image = random.sample(golden_ref_images,1)[0] #pair each sample with a golden reference image 
    shutil.copy(os.path.join(no_defect_dir, golden_image), os.path.join(b_dir, image))

In [ ]:
#Create train, test, validation text files 
#The datasets are fairly small so we will make test and validation the same

train_split = 0.7 #percent to put in train set 

samples = os.listdir(a_dir)
random.shuffle(samples)

train = []
test = []

#Split classes into train and test samples
for cls in mvtec_object_classes:
    cls_images = [image for image in samples if cls in image]
    index_split = int(len(cls_images) * train_split)
    train+=cls_images[0:index_split]
    test+= cls_images[index_split:]

test_labels = "\n".join(test)
train_labels = "\n".join(train)

#Write out test,val,train lists
with open(os.path.join(list_dir, "test.txt"), "w+") as file:
    file.write(test_labels)

with open(os.path.join(list_dir, "val.txt"), "w+") as file:
    file.write(test_labels)

with open(os.path.join(list_dir, "train.txt"), "w+") as file:
    file.write(train_labels)         

### 2.4 Verify Dataset Format <a class="anchor" id="head-2-4"></a>

In [ ]:
import os
import shutil
def verify_dataset(data_dir):

    num_images_A = len(os.listdir(a_dir))
    num_images_B = len(os.listdir(b_dir))
    num_labels = len(os.listdir(label_dir))
    num_train_images = sum(1 for _ in open(os.path.join(list_dir, "train.txt")))
    num_val_images = sum(1 for _ in open(os.path.join(list_dir, "val.txt")))
    num_test_images = sum(1 for _ in open(os.path.join(list_dir, "test.txt")))
    print("Number of images in the train/val/test set. {}".format(num_images_A))
    print("Number of compare images in the train/val/test set. {}".format(num_images_B))
    print("Number of labels in the train/val/test set. {}".format(num_labels))
    print("Number of train instances. {}".format(num_train_images))
    print("Number of val instances. {}".format(num_val_images))
    print("Number of test instances. {}".format(num_test_images))

# verify downloaded dataset
verify_dataset(data_dir=formatted_path)

In [ ]:
# Verify
!ls -l $HOST_DATA_DIR/$FORMATTED_DATA_DIR

# 3. Download the pretrained model from NGC <a class="anchor" id="head-3"></a>

Download the pretrained model from NGC. The NGC CLI is used to get the data and model. For more details, go to https://ngc.nvidia.com and click the SETUP on the navigation bar.

In [ ]:
# Installing NGC CLI on the local machine.
## Download and install
import os
import platform

if platform.machine() == "x86_64":
    os.environ["CLI"]="ngccli_linux.zip"
else:
    os.environ["CLI"]="ngccli_arm64.zip"


# # Remove any previously existing CLI installations
!rm -rf $HOST_RESULTS_DIR/ngccli/*
!wget "https://ngc.nvidia.com/downloads/$CLI" -P $HOST_RESULTS_DIR/ngccli
!unzip -u "$HOST_RESULTS_DIR/ngccli/$CLI" -d $HOST_RESULTS_DIR/ngccli/
!rm $HOST_RESULTS_DIR/ngccli/*.zip
os.environ["PATH"]="{}/ngccli/ngc-cli:{}".format(os.getenv("HOST_RESULTS_DIR", ""), os.getenv("PATH", ""))

In [ ]:
!mkdir -p $HOST_RESULTS_DIR/pretrained

In [ ]:
!ngc registry model list nvidia/tao/pretrained_fan_classification_nvimagenet*

In [ ]:
!ngc registry model download-version "nvidia/tao/pretrained_fan_classification_nvimagenet:fan_base_hybrid_nvimagenet" --dest $HOST_RESULTS_DIR/pretrained

In [ ]:
print("Check that model is downloaded into dir.")
!ls -l $HOST_RESULTS_DIR/pretrained/pretrained_fan_classification_nvimagenet_vfan_base_hybrid_nvimagenet

## 4. Provide training specification <a class="anchor" id="head-4"></a>

See the TAO documentation on Visual ChangeNet-Segmentation to determine the parameters that are configurable.

In [ ]:
spec = """
encryption_key: tlt_encode
task: segment
train:
  resume_training_checkpoint_path: null
  pretrained_model_path: null
  segment:
    loss: "ce"
    weights: [0.5, 0.5, 0.5, 0.8, 1.0]
  num_epochs: 30
  num_nodes: 1
  validation_interval: 1
  checkpoint_interval: 1
  optim:
    lr: 0.0002
    optim: "adamw"
    policy: "linear" 
    momentum: 0.9
    weight_decay: 0.01
results_dir: "/results"
model:
  backbone:
    type: "fan_base_16_p4_hybrid"
    pretrained_backbone_path: /results/pretrained/pretrained_fan_classification_nvimagenet_vfan_base_hybrid_nvimagenet/fan_base_hybrid_nvimagenet.pth
dataset:
  segment:
    dataset: "CNDataset"
    root_dir: /data/formatted_dataset
    data_name: "custom"
    label_transform: "norm"
    batch_size: 8
    workers: 1
    multi_scale_train: True
    multi_scale_infer: False
    num_classes: 2
    img_size: 256
    image_folder_name: "A"
    change_image_folder_name: 'B'
    list_folder_name: 'list'
    annotation_folder_name: "label"
    train_split: "train"
    validation_split: "val"
    test_split: 'test'
    predict_split: 'test'
    label_suffix: .png
    augmentation: 
      random_flip:
        vflip_probability: 0.5
        hflip_probability: 0.5
        enable: True
      random_rotate:
        rotate_probability: 0.5
        angle_list: [90, 180, 270]
        enable: True
      random_color:
        brightness: 0.3
        contrast: 0.3
        saturation: 0.3
        hue: 0.3
        enable: True
      with_scale_random_crop:
        enable: True
      with_random_crop: True
      with_random_blur: True
evaluate: 
  results_dir: "${results_dir}/evaluate"
  checkpoint: "${results_dir}/train/changenet.pth"
  trt_engine: "${gen_trt_engine.trt_engine}"
  batch_size: ${dataset.segment.batch_size}
  vis_after_n_batches: 1
inference:
  results_dir: "${results_dir}/inference"
  checkpoint: "${results_dir}/train/changenet.pth"
  trt_engine: "${gen_trt_engine.trt_engine}"
  batch_size: ${dataset.segment.batch_size}
  vis_after_n_batches: 1
export:
  results_dir: "${results_dir}/export"
  gpu_id: 0
  checkpoint: "${results_dir}/train/changenet.pth"
  onnx_file: "${export.results_dir}/changenet.onnx"
  input_width: 256
  input_height: 256
  batch_size: ${dataset.segment.batch_size}
gen_trt_engine:
  results_dir: "${results_dir}/gen_trt_engine"
  gpu_id: 0
  onnx_file: "${export.onnx_file}"
  trt_engine: "${gen_trt_engine.results_dir}/changenet.trt"
  batch_size: ${dataset.segment.batch_size}
  tensorrt:
    data_type: FP32
    workspace_size: 1024
    min_batch_size: 1
    opt_batch_size: 10
    max_batch_size: 10
    """

In [ ]:
#Write out the spec file 
with open(os.path.join(os.environ["HOST_SPECS_DIR"], "experiment.yaml"), "w+") as file:
    file.write(spec)

## 5. Run TAO training <a class="anchor" id="head-5"></a>
* Training will take 20-30 minutes depending on your GPU. 

* To replicate baseline numbers accurately, the model must be trained for 30 epochs.

In [ ]:
# NOTE: The following paths are set from the perspective of the TAO Docker.
# The data is saved here
%env DATA_DIR = /data/$FORMATTED_DATA_DIR
%env MODEL_DIR = /model
%env SPECS_DIR = /specs
%env RESULTS_DIR = /results
%env NUM_EPOCHS = 30
%env BACKBONE_PATH = /results/pretrained/pretrained_fan_classification_nvimagenet_vfan_base_hybrid_nvimagenet/fan_base_hybrid_nvimagenet.pth

### 5.1 Train Visual ChangeNet-Segmentation model

Train a Visual ChangeNet-Segmentation model with a pretrained FAN backbone.

In [ ]:
print("Train model")
!tao model visual_changenet train \
                  -e $SPECS_DIR/experiment.yaml \
                    train.num_epochs=$NUM_EPOCHS \
                    dataset.segment.root_dir=$DATA_DIR \
                    model.backbone.pretrained_backbone_path=$BACKBONE_PATH

In [ ]:
print('Model checkpoints:')
!ls -ltrh $HOST_RESULTS_DIR/train

In [ ]:
# You can set NUM_EPOCH to the epoch corresponding to any saved checkpoint
# %env NUM_EPOCH=029

# Get the name of the checkpoint corresponding to your set epoch
# tmp=!ls $HOST_RESULTS_DIR/train/*.pth | grep epoch_$NUM_EPOCH
# %env CHECKPOINT={tmp[0]}

# Or get the latest checkpoint
os.environ["CHECKPOINT"] = os.path.join(os.getenv("HOST_RESULTS_DIR"), "train/changenet_model_segment_latest.pth")

print('Rename a trained model: ')
print('---------------------')
!cp $CHECKPOINT $HOST_RESULTS_DIR/train/changenet.pth
!ls -ltrh $HOST_RESULTS_DIR/train/changenet.pth

## 6. Evaluate trained models <a class="anchor" id="head-6"></a>
Evaluate trained model.

For Visual ChangeNet-Segmentation model evaluation, the following metrics are preferred:

* Overall Accuracy.
* mIoU: Intersection over Union (IoU) for all classes.

Metrics for Precision, Recall, IoU, and F1 score are provided for each individual class and averaged over all classes. 

In [ ]:
!tao model visual_changenet evaluate \
                   -e $SPECS_DIR/experiment.yaml \
                    evaluate.checkpoint=$RESULTS_DIR/train/changenet.pth \
                    dataset.segment.root_dir=$DATA_DIR

## 7. Visualize Inference <a class="anchor" id="head-7"></a>
In this section, run the Visual ChangeNet-Segmentation inference tool to generate inferences with the trained models and save the results under `$RESULTS_DIR`. And visualise a few sample outputs along with their corresponding input images. 

In [ ]:
!tao model visual_changenet inference \
                   -e $SPECS_DIR/experiment.yaml \
                    inference.checkpoint=$RESULTS_DIR/train/changenet.pth \
                    dataset.segment.root_dir=$DATA_DIR

In [ ]:
# Simple grid visualizer
!pip3 install "matplotlib>=3.3.3, <4.0"
import matplotlib.pyplot as plt
import os
from math import ceil
valid_image_ext = ['.jpg']

def visualize_images(output_path, num_cols=4, num_images=10):
    num_rows = int(ceil(float(num_images) / float(num_cols)))
    f, axarr = plt.subplots(num_rows, num_cols, figsize=[80,30])
    f.tight_layout()
    a = [os.path.join(output_path, image) for image in os.listdir(output_path) 
         if os.path.splitext(image)[1].lower() in valid_image_ext]
    for idx, img_path in enumerate(a[:num_images]):
        col_id = idx % num_cols
        row_id = idx // num_cols
        img = plt.imread(img_path)
        axarr[row_id, col_id].imshow(img) 

In [ ]:
# Visualizing the sample images.
IMAGE_DIR = os.path.join(os.environ['HOST_RESULTS_DIR'], "inference")
COLS = 2 # number of columns in the visualizer grid.
IMAGES = 8 # number of images to visualize.

visualize_images(IMAGE_DIR, num_cols=COLS, num_images=IMAGES)

## 8. Deploy <a class="anchor" id="head-8"></a>
Export the model to encrypted ONNX model.

In [ ]:
!mkdir -p $HOST_RESULTS_DIR/export

In [ ]:
!tao model visual_changenet export \
                    -e $SPECS_DIR/experiment.yaml \
                        export.checkpoint=$RESULTS_DIR/train/changenet.pth \
                        export.onnx_file=$RESULTS_DIR/export/changenet.onnx

In [ ]:
# # Uncomment this cell to export an onnx file with dynamic batching enabled for
# # integration with trtexec and deepstream.
# # Export the model with export.batch_size=-1 for dynamic batching.
# ! tao model visual_changenet export \
#                     -e $SPECS_DIR/experiment.yaml \
#                     export.checkpoint=$RESULTS_DIR/train/changenet.pth \
#                     export.batch_size=-1 \
#                     export.results_dir=$RESULTS_DIR/export_dynamic \
#                     export.onnx_file=$RESULTS_DIR/export_dynamic/changenet.onnx
                            

# # Profiling the exported model via trtexec.
# ! tao deploy trtexec --onnx=$RESULTS_DIR/export_dynamic/changenet.onnx \
#                      --minShapes=input0:1x3x512x128,input1:1x3x256x256 \
#                      --optShapes=input0:8x3x512x128,input1:8x3x256x256 \
#                      --maxShapes=input0:16x3x512x128,input1:16x3x256x256 \
#                      --fp16 \
#                      --saveEngine=$RESULTS_DIR/export_dynamic/changenet-fp16.engine

In [ ]:
print('Exported model:')
print('------------')
!ls -lth $HOST_RESULTS_DIR/export

In [ ]:
!tao deploy visual_changenet gen_trt_engine \
            -e $SPECS_DIR/experiment.yaml \
                gen_trt_engine.onnx_file=$RESULTS_DIR/export/changenet.onnx \
                gen_trt_engine.trt_engine=$RESULTS_DIR/gen_trt_engine/changenet.trt

In [ ]:
!tao deploy visual_changenet inference \
            -e $SPECS_DIR/experiment.yaml \
                inference.trt_engine=$RESULTS_DIR/gen_trt_engine/changenet.trt \
                inference.results_dir=$RESULTS_DIR/trt_inference \
                dataset.segment.root_dir=$DATA_DIR

In [ ]:
!tao deploy visual_changenet evaluate \
            -e $SPECS_DIR/experiment.yaml \
                evaluate.trt_engine=$RESULTS_DIR/gen_trt_engine/changenet.trt \
                evaluate.results_dir=$RESULTS_DIR/trt_evaluate \
                dataset.segment.root_dir=$DATA_DIR

In [ ]:
# Visualizing the sample images.
IMAGE_DIR = os.path.join(os.environ['HOST_RESULTS_DIR'], "trt_inference")
COLS = 2 # number of columns in the visualizer grid.
IMAGES = 4 # number of images to visualize.

visualize_images(IMAGE_DIR, num_cols=COLS, num_images=IMAGES)

This notebook has come to an end.